In [1]:
from socket import *
import json
import base64

In [2]:
class Livro:
    def __init__(self,nome,conteudo):
        self.nome=nome
        self.conteudo=conteudo

class Message():
    def __init__(self, tipoMensagem, idRequisicao, idOperacao, livro):
        self.tipoMensagem = tipoMensagem
        self.idRequisicao=idRequisicao
        self.idOperacao = idOperacao
        self.livro = livro



In [50]:
def getRequest(connectionSocket):
    BUFF_SIZE = 4096 # 4 KiB
    data = b''
    while True:
        part = connectionSocket.recv(BUFF_SIZE)
        data += part
        if len(part) < BUFF_SIZE:
            # either 0 or end of data
            break
    return data

def sendReply(connectionSocket, mensagem):
  
    m={'tipoMensagem':mensagem.tipoMensagem,'idRequisicao':mensagem.idRequisicao,'idOperacao':mensagem.idOperacao,'livro': {"nome": mensagem.livro.nome,"conteudo": mensagem.livro.conteudo}}
    
    r = json.dumps(m)
    print(r)
    print("enviando")
    
    connectionSocket.send(r.encode('UTF8'))


In [ ]:

serverPort = 2111
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(('', serverPort))
serverSocket.listen(1)
print('The server is ready to receive')
while True:
    #cria o socket pro cliente e estabelece a conexão
    connectionSocket, addr = serverSocket.accept()
 
    #sentence = connectionSocket.recv(8096)
    
    dado = json.loads(getRequest(connectionSocket).decode())
    print(dado)
    l = Livro(dado['livro']['nome'],dado['livro']['conteudo'])
    m = Message(dado['tipoMensagem'],dado['idRequisicao'],dado['idOperacao'],l)

    if(m.idOperacao==1):
        fileData = base64.b64decode(l.conteudo)
        # make file
        newFile = open("livros/"+l.nome+".pdf", "wb")
        # write to file
        newFile.write(fileData) 
        m.livro = l
        sendReply(connectionSocket, m)
    else:
        newFile = open("livros/"+l.nome+".pdf","rb")
        BUFF_SIZE = 4096 # 4 KiB
        data = b''
        byte = newFile.read(1)
        while byte!= b'':
            data += byte
            byte = newFile.read(1)
        print("Estou aqui")    
        fileData = base64.b64encode(data)
        l.conteudo = str(fileData,"utf-8")
        sendReply(connectionSocket, m)
    
    connectionSocket.close()
    
    

    

The server is ready to receive
{'tipoMensagem': 0, 'idRequisicao': 0, 'idOperacao': 2, 'livro': {'nome': 'b', 'conteudo': 'vazio'}}
Estou aqui
{"tipoMensagem": 0, "idRequisicao": 0, "idOperacao": 2, "livro": {"nome": "b", "conteudo": "..."}}
enviando


{'tipoMensagem': 0, 'idRequisicao': 0, 'idOperacao': 2, 'livro': {'nome': 'b', 'conteudo': 'vazio'}}
Estou aqui
{"tipoMensagem": 0, "idRequisicao": 0, "idOperacao": 2, "livro": {"nome": "b", "conteudo": "..."}}
enviando


In [41]:
x={'b':{'s':'c'}}
r=json.dumps(x)
print(r)
y=json.loads(r)
print(y)

{"b": {"s": "c"}}
{'b': {'s': 'c'}}
